## Starting with a Linear Regression Model

| Model Features | --- | --- | --- |---|
| --- | --- |--- | --- |---|
| **Numeric** | --- |--- | --- |---|
| Lot Area | Overall Quality | Overall Condition | Year Remod/Add | Total SF |
| Total Bathrooms | Fireplaces | Garage Area | --- | --- |
| --- | --- |--- | --- |---|
| **Catagorical** | --- | --- | --- |---|
| MS SubClass | MS Zoning | Lot Config | Neighborhood | Condition 1|
| Bldg Type | House Style |Exterior 1st  | Mas Vnr Type | Exter Qual |
| Exter Cond | Foundation | Bsmt Qual | BsmtFin Type 1 | Heating QC |
| Kitchen Qual | Sale Type |--- | --- |---|
| --- | --- |--- | --- |---|


In [1]:
#importing libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn import metrics
#import re

# Data Dictionary - [Link](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) 

In [2]:
#importing clean v5_data
housing_data = pd.read_csv("../datasets/complete_training_data.csv")
#importing testing data
testing_data = pd.read_csv("../datasets/complete_kaggle_test.csv")

In [3]:
housing_data.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [4]:
ms_subclass_dummies = pd.get_dummies(housing_data["MS SubClass"],prefix="SubClass")
ms_subclass_dummies.drop(columns=["SubClass_150","SubClass_40"],inplace=True)

ms_zoning_dummies = pd.get_dummies(housing_data["MS Zoning"])
ms_zoning_dummies.drop(columns=["A (agr)","I (all)"],inplace=True)

lot_config_dummies = pd.get_dummies(housing_data["Lot Config"])
lot_config_dummies.drop(columns=["FR3"],inplace=True)

neighborhood_dummies = pd.get_dummies(housing_data["Neighborhood"])
neighborhood_dummies.drop(columns=["Landmrk","GrnHill","Greens","Blueste"],inplace=True)

condition_1_dummies = pd.get_dummies(housing_data["Condition 1"])
condition_1_dummies.drop(columns=["RRNe","RRNn"],inplace=True)

bldg_type_dummies = pd.get_dummies(housing_data["Bldg Type"])
bldg_type_dummies.drop(columns="Duplex",inplace=True)

house_style_dummies = pd.get_dummies(housing_data["House Style"])
house_style_dummies.drop(columns=["2.5Fin"],inplace=True)

exterior_1_dummies = pd.get_dummies(housing_data["Exterior 1st"],prefix="Ex")
exterior_1_dummies.drop(columns=["Ex_AsphShn","Ex_CBlock","Ex_ImStucc","Ex_Stone","Ex_BrkComm"],inplace=True)

mas_vnr_type_dummies = pd.get_dummies(housing_data["Mas Vnr Type"],prefix="MsV")
mas_vnr_type_dummies.drop(columns=["MsV_BrkCmn"],inplace=True)

exter_qual_dummies = pd.get_dummies(housing_data["Exter Qual"],prefix="ExQ")
exter_qual_dummies.drop(columns="ExQ_Fa",inplace=True)

exter_cond_dummies = pd.get_dummies(housing_data["Exter Cond"],prefix="ExC")
#might have issues with Ex not being in training data
exter_cond_dummies.drop(columns=["ExC_Po"],inplace=True)

foundation_dummies = pd.get_dummies(housing_data["Foundation"])
foundation_dummies.drop(columns=["Stone","Wood"],inplace=True)

bsmt_qual_dummies = pd.get_dummies(housing_data["Bsmt Qual"],prefix="BsQ")
bsmt_qual_dummies.drop(columns=["BsQ_Po","BsQ_Fa"],inplace=True)

bsmtfin_type1_dummies = pd.get_dummies(housing_data["BsmtFin Type 1"])
bsmtfin_type1_dummies.drop(columns="LwQ",inplace=True)

heatingqc_dummies = pd.get_dummies(housing_data["Heating QC"],prefix="HtQC")
heatingqc_dummies.drop(columns=["HtQC_Po","HtQC_Fa"],inplace=True)

kitchen_qual_dummies = pd.get_dummies(housing_data["Kitchen Qual"],prefix="Kit")
kitchen_qual_dummies.drop(columns="Kit_Fa",inplace=True)

garage_type_dummies = pd.get_dummies(housing_data["Garage Type"])
garage_type_dummies.drop(columns="CarPort",inplace=True)

sale_type_dummies = pd.get_dummies(housing_data["Sale Type"])
sale_type_dummies.drop(columns=["Con","Oth","ConLI","ConLw"],inplace=True)

In [5]:
xvars = ["Lot Area","Overall Qual","Overall Cond","Year Remod/Add","Total SF","Total Bathrooms","Fireplaces","Garage Area"]

In [6]:
X = housing_data[xvars]
y = housing_data["SalePrice"]

In [7]:
#creating matrix for xvars and dummy cells
X = pd.concat([X,
ms_subclass_dummies,
ms_zoning_dummies,
lot_config_dummies,
neighborhood_dummies,
condition_1_dummies,
bldg_type_dummies,
house_style_dummies,
exterior_1_dummies,
mas_vnr_type_dummies,
exter_qual_dummies,
exter_cond_dummies,
foundation_dummies,
bsmt_qual_dummies,
bsmtfin_type1_dummies,
heatingqc_dummies,
kitchen_qual_dummies,
garage_type_dummies,
sale_type_dummies
              ], axis=1)
X.shape

(2023, 121)

In [8]:
linreg = LinearRegression()

In [9]:
#spliting the data 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=2020)

In [10]:
X_train.shape

(1618, 121)

In [11]:
linreg.fit(X_train,y_train)

LinearRegression()

In [12]:
#looking at 5 part cross validation on traing data. The R2 is quite good at 0.88
linreg_scores = cross_val_score(linreg, X_train, y_train, cv=5)
linreg_scores.mean()

0.9071408173362407

In [13]:
preds_train = linreg.predict(X_train)
preds_test = linreg.predict(X_test)

In [14]:
display(linreg.score(X_train,y_train))
display(linreg.score(X_test,y_test))

0.924066158446986

0.8861769204534597

In [15]:
X_train_sm = X_train
X_train_sm = sm.add_constant(X_train_sm)
y_train_sm = y_train

In [16]:
sm_model = sm.OLS(y_train_sm,X_train_sm).fit()

In [17]:
results_summary = sm_model.summary()

In [18]:
results_summary.tables[0]

Dep. Variable:,SalePrice,R-squared:,0.924
Model:,OLS,Adj. R-squared:,0.918
Method:,Least Squares,F-statistic:,151.8
Date:,"Sun, 16 Aug 2020",Prob (F-statistic):,0.00
Time:,18:09:11,Log-Likelihood:,-18472.
No. Observations:,1618,AIC:,3.719e+04
Df Residuals:,1497,BIC:,3.784e+04
Df Model:,120,,
Covariance Type:,nonrobust,,


In [19]:
results_as_html = results_summary.tables[1].as_html()
coef = pd.read_html(results_as_html, header=0, index_col=0)[0]

In [20]:
coef.head(9)

,coef,std err,t,P>|t|,[0.025,0.975]
const,-227400.0000,89300.000,-2.546,0.011,-403000.000,-52200.000
Lot Area,2.0555,0.249,8.256,0.000,1.567,2.544
Overall Qual,8275.5394,900.811,9.187,0.000,6508.553,10000.000
Overall Cond,6113.1818,744.658,8.209,0.000,4652.498,7573.865
Year Remod/Add,113.7203,49.510,2.297,0.022,16.604,210.837
Total SF,35.1173,1.535,22.881,0.000,32.107,38.128
Total Bathrooms,6946.0567,1236.110,5.619,0.000,4521.366,9370.748
Fireplaces,5880.5177,1246.738,4.717,0.000,3434.978,8326.057
Garage Area,23.3395,4.901,4.762,0.000,13.726,32.953


### Conclusions from modeling

The train score for this model is 0.90 (test score is 0.878). I think that this model could fit the data better but it is a good start set as the bench mark. This model is also very interperable which is a nice feature.

### Calculating sales prices in training data (need to output .csv w/ header Id,SalePrice)

In [21]:
X_kaggle = testing_data[xvars]
kaggle_ms_subclass_dummies = pd.get_dummies(testing_data["MS SubClass"],prefix="SubClass")
kaggle_ms_subclass_dummies.drop(columns=["SubClass_40"],inplace=True)

kaggle_ms_zoning_dummies = pd.get_dummies(testing_data["MS Zoning"])
kaggle_ms_zoning_dummies.drop(columns=["I (all)"],inplace=True)

kaggle_lot_config_dummies = pd.get_dummies(testing_data["Lot Config"])
kaggle_lot_config_dummies.drop(columns=["FR3"],inplace=True)

kaggle_neighborhood_dummies = pd.get_dummies(testing_data["Neighborhood"])
kaggle_neighborhood_dummies.drop(columns=["Greens","Blueste"],inplace=True)

kaggle_condition_1_dummies = pd.get_dummies(testing_data["Condition 1"])
kaggle_condition_1_dummies.drop(columns=["RRNe","RRNn"],inplace=True)

kaggle_bldg_type_dummies = pd.get_dummies(testing_data["Bldg Type"])
kaggle_bldg_type_dummies.drop(columns="Duplex",inplace=True)

kaggle_house_style_dummies = pd.get_dummies(testing_data["House Style"])
kaggle_house_style_dummies.drop(columns=["2.5Fin"],inplace=True)

kaggle_exterior_1_dummies = pd.get_dummies(testing_data["Exterior 1st"],prefix="Ex")
kaggle_exterior_1_dummies.drop(columns=["Ex_AsphShn","Ex_BrkComm","Ex_PreCast"],inplace=True)

kaggle_mas_vnr_type_dummies = pd.get_dummies(testing_data["Mas Vnr Type"],prefix="MsV")
kaggle_mas_vnr_type_dummies.drop(columns=["MsV_BrkCmn","MsV_CBlock"],inplace=True)

kaggle_exter_qual_dummies = pd.get_dummies(testing_data["Exter Qual"],prefix="ExQ")
kaggle_exter_qual_dummies.drop(columns="ExQ_Fa",inplace=True)

kaggle_exter_cond_dummies = pd.get_dummies(testing_data["Exter Cond"],prefix="ExC")
kaggle_exter_cond_dummies.drop(columns=["ExC_Po"],inplace=True)

kaggle_foundation_dummies = pd.get_dummies(testing_data["Foundation"])
kaggle_foundation_dummies.drop(columns=["Stone","Wood"],inplace=True)

kaggle_bsmt_qual_dummies = pd.get_dummies(testing_data["Bsmt Qual"],prefix="BsQ")
kaggle_bsmt_qual_dummies.drop(columns=["BsQ_Po","BsQ_Fa"],inplace=True)

kaggle_bsmtfin_type1_dummies = pd.get_dummies(testing_data["BsmtFin Type 1"])
kaggle_bsmtfin_type1_dummies.drop(columns="LwQ",inplace=True)

kaggle_heatingqc_dummies = pd.get_dummies(testing_data["Heating QC"],prefix="HtQC")
kaggle_heatingqc_dummies.drop(columns=["HtQC_Fa"],inplace=True)

kaggle_kitchen_qual_dummies = pd.get_dummies(testing_data["Kitchen Qual"],prefix="Kit")
kaggle_kitchen_qual_dummies.drop(columns=["Kit_Fa","Kit_Po"],inplace=True)

kaggle_garage_type_dummies = pd.get_dummies(testing_data["Garage Type"])
kaggle_garage_type_dummies.drop(columns="CarPort",inplace=True)

kaggle_sale_type_dummies = pd.get_dummies(testing_data["Sale Type"])
kaggle_sale_type_dummies.drop(columns=["Con","Oth","ConLI","ConLw","VWD"],inplace=True)

In [22]:
X_kaggle = pd.concat([X_kaggle,
kaggle_ms_subclass_dummies,
kaggle_ms_zoning_dummies,
kaggle_lot_config_dummies,
kaggle_neighborhood_dummies,
kaggle_condition_1_dummies,
kaggle_bldg_type_dummies,
kaggle_house_style_dummies,
kaggle_exterior_1_dummies,
kaggle_mas_vnr_type_dummies,
kaggle_exter_qual_dummies,
kaggle_exter_cond_dummies,
kaggle_foundation_dummies,
kaggle_bsmt_qual_dummies,
kaggle_bsmtfin_type1_dummies,
kaggle_heatingqc_dummies,
kaggle_kitchen_qual_dummies,
kaggle_garage_type_dummies,
kaggle_sale_type_dummies
              ], axis=1)
X_kaggle.shape

(878, 121)

In [23]:
#finding differences between train and testing data columns. They need to match for fit to make sense
for i,x in enumerate(X_train.columns):
    if X_kaggle.columns[i] != x:
        print(i,x)
    else:
        pass

In [24]:
price_X_testing = linreg.predict(X_kaggle)
testing_data["SalePrice"] = price_X_testing
testing_data.head(1)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Total SF x Garage Area,Overall Qual^2,Year Remod/Add x Total SF,Total SF^2,Overall Qual x Garage Area,Overall Qual x Total Bathrooms,Total SF x Total Bathrooms,Overall Qual x Year Remod/Add,Total Bathrooms x Garage Area,SalePrice
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,1297120,36,5748600,8690704,2640,12.0,5896.0,11700,880.0,166520.781167


In [26]:
mlr_27part_fit = testing_data[["Id","SalePrice"]]
mlr_27part_fit.to_csv("../datasets/mlr_27part_fit.csv",index=False)